## Análise de Sentimentos

In [2]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", device=0)
result = sentiment_analysis("Esse produto é maravilhoso!")
print(result)


[{'label': '5 stars', 'score': 0.8916902542114258}]


O modelo `nlptown/bert-base-multilingual-uncased-sentiment` é ajustado para **análise de sentimentos em 5 níveis**, com as seguintes labels:

- **1 star**: Sentimento muito negativo
- **2 stars**: Sentimento negativo
- **3 stars**: Sentimento neutro
- **4 stars**: Sentimento positivo
- **5 stars**: Sentimento muito positivo


## NER

In [106]:
from transformers import pipeline

# Configura o pipeline para classificação de token com truncamento
ner = pipeline("token-classification", model="pierreguillou/ner-bert-base-cased-pt-lenerbr", device=0)
entities = ner("O presidente da França é Emmanuel Macron. E a chanceler da Alemanha é Angela Merkel. Mas minha casa é em São Paulo.")
print(entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}, {'entity': 'B-PESSOA', 'score': np.float32(0.9997147), 'index': 6, 'word': 'Emma', 'start': 25, 'end': 29}, {'entity': 'B-PESSOA', 'score': np.float32(0.99970573), 'index': 7, 'word': '##nu', 'start': 29, 'end': 31}, {'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 8, 'word': '##el', 'start': 31, 'end': 33}, {'entity': 'I-PESSOA', 'score': np.float32(0.9997819), 'index': 9, 'word': 'Mac', 'start': 34, 'end': 37}, {'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 10, 'word': '##ron', 'start': 37, 'end': 40}, {'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}, {'entity': 'B-PESSOA', 'score': np.float32(0.9995801), 'index': 18, 'word': 'Angel', 'start': 70, 'end': 75}, {'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 19, 'word': '##a', 'start': 75, 'end': 76}, {'entity': 'I-PE

In [107]:
from typing import List, Dict
import numpy as np

def group_entities(entities: List[Dict]) -> List[Dict]:
    grouped_entities = []
    current_group = None

    for entity in entities:
        if current_group and entity['start'] == current_group['end']:
            # Append to the current group
            current_group['word'] += entity['word'].replace("##", "")
            current_group['end'] = entity['end']
            current_group['score'] = min(current_group['score'], entity['score'])  # Atualiza o score
        else:
            # Finaliza o grupo atual e inicia um novo
            if current_group:
                grouped_entities.append(current_group)
            current_group = entity.copy()  # Inicia um novo grupo com a entidade atual
    
    # Adiciona o último grupo
    if current_group:
        grouped_entities.append(current_group)

    return grouped_entities

In [108]:
result = group_entities(entities)

# Resultado agrupado
for item in result:
    print(item)

{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}
{'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 6, 'word': 'Emmanuel', 'start': 25, 'end': 33}
{'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 9, 'word': 'Macron', 'start': 34, 'end': 40}
{'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}
{'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 18, 'word': 'Angela', 'start': 70, 'end': 76}
{'entity': 'I-PESSOA', 'score': np.float32(0.9997882), 'index': 20, 'word': 'Merkel', 'start': 77, 'end': 83}
{'entity': 'B-LOCAL', 'score': np.float32(0.9991271), 'index': 28, 'word': 'São', 'start': 105, 'end': 108}
{'entity': 'I-LOCAL', 'score': np.float32(0.99898416), 'index': 29, 'word': 'Paulo', 'start': 109, 'end': 114}


In [120]:
final = []

for pos, item in enumerate(result):
    if 'B' in item['entity']:
        new_group = []
        new_group.append(item)
        i = 1
        while 'I' in result[result.index(item)+i]['entity']:
            if pos+i+1 < len(result):
                new_group.append(result[result.index(item)+i])
                i += 1
            else:
                new_group.append(result[result.index(item)+i])
                break
    if 'I' in item['entity']:
        continue
    print(new_group)

[{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}]
[{'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 6, 'word': 'Emmanuel', 'start': 25, 'end': 33}, {'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 9, 'word': 'Macron', 'start': 34, 'end': 40}]
[{'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}]
[{'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 18, 'word': 'Angela', 'start': 70, 'end': 76}, {'entity': 'I-PESSOA', 'score': np.float32(0.9997882), 'index': 20, 'word': 'Merkel', 'start': 77, 'end': 83}]
[{'entity': 'B-LOCAL', 'score': np.float32(0.9991271), 'index': 28, 'word': 'São', 'start': 105, 'end': 108}, {'entity': 'I-LOCAL', 'score': np.float32(0.99898416), 'index': 29, 'word': 'Paulo', 'start': 109, 'end': 114}]


## Sumarização 

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carrega o tokenizer e o modelo na GPU
tokenizer = T5Tokenizer.from_pretrained("PamelaBorelli/flan-t5-base-summarization-pt-br")
model = T5ForConditionalGeneration.from_pretrained("PamelaBorelli/flan-t5-base-summarization-pt-br").to("cuda")

# Texto de entrada e tokenização
input_text = (
    "O corpo está mais propenso a sentir dores com exercícios de alta intensidade | Foto: Getty Images O problema está em saber identificar qual é qual. "
    "'Em algumas situações, é difícil diferenciar uma da outra', reconhece Juan Francisco Marco, professor do Centro de Ciência do Esporte, Treinamento e Fitness Alto Rendimento, na Espanha. "
    "'A dor boa é aquela que associamos ao exercício físico, que não limita (o movimento) e permite continuar (a se exercitar) até o momento em que o músculo fica realmente esgotado e não trabalha mais', explica. "
    "É importante detectar qual é o tipo de dor que você está sentindo, para evitar ter problemas mais sérios | Foto: Getty Images Para Francisco Sánchez Diego, diretor do centro de treinamento Corpore 10, 'a dor boa se sente no grupo muscular que você trabalhou, tanto durante o treinamento como nos dias seguintes'."
)

# Move os input_ids para a GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# Geração do resumo
outputs = model.generate(input_ids)

# Decodificação e impressão do resultado
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/gui/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


O problema é difcil diferenciar uma da outr
